In [1]:
##READY TO CONVERT TO POINT TO PATCH with BRDF

import numpy as np
from scipy.integrate import dblquad
import sparrowpy.geometry as geom
from numpy.polynomial.legendre import leggauss
import pyfar as pf
import sparrowpy as sp
from scipy.spatial import cKDTree

sh_order = 2
scattering_directions = pf.samplings.sph_lebedev(sh_order=sh_order)

scattering_directions.weights = pf.samplings.calculate_sph_voronoi_weights(scattering_directions)
scattering_directions = scattering_directions[scattering_directions.z>0]

brdf_data = sp.brdf.create_from_scattering(
scattering_directions, scattering_directions, pf.FrequencyData(0, [500]))


patch_points_1 = np.array([[-1.        , -1.        ,  0.        ],
                      [-0.33333333, -1.        ,  0.        ],
                      [-0.33333333, -0.33333333,  0.        ],
                      [-1.        , -0.33333333,  0.        ]])

patch_points_2 = np.array([[-1.        , -0.33333333        ,  0.        ],
                      [-0.33333333, -0.33333333        ,  0.        ],
                      [-0.33333333, 0.33333333,  0.        ],
                      [-1.        , 0.33333333,  0.        ]])
                      
src_point_tmp = np.array([0.70710678, 0.        , 0.70710678])
src_point = pf.Coordinates(src_point_tmp[0],src_point_tmp[1],src_point_tmp[2],domain='cart')
rcv_point = pf.Coordinates(src_point.azimuth + np.pi, src_point.elevation, src_point.radius, domain='sph')


#working factor with BRDF
patch_pts = patch_points_1
source_pos = src_point.cartesian[0]
receiver_pos = rcv_point.cartesian[0]

# Calculate the center point of patch_points_2
patch_center = np.mean(patch_pts, axis=0)

# Shift the coordinate into the patch_points_2 center point
shifted_coord = scattering_directions.cartesian + patch_center
tree_i = cKDTree(scattering_directions.cartesian )  # shape (930*930,3)

omnidirectional = True
L_i_data =  np.zeros(len(shifted_coord))
if omnidirectional:
    factor = 1/(4*np.pi)
    L_i_data =  np.full(len(shifted_coord), factor)
else:
    # load directivity data?
    pass

N=3        
p0     = patch_pts[0]
du_vec = patch_pts[1] - p0  # vector along u
dv_vec = patch_pts[3] - p0  # vector along v

# Precompute patch area and ensure normal orientation
cross = np.cross(du_vec, dv_vec)
area  = np.linalg.norm(cross)
n_p   = cross / area
# Flip normal if it faces away from source
patch_center = p0 + 0.5*(du_vec + dv_vec)
if np.dot(n_p, source_pos - patch_center) < 0:
    n_p = -n_p

# Setup Gauss–Legendre nodes & weights on [0,1]
u_nodes, u_wts = leggauss(N)
v_nodes, v_wts = leggauss(N)

total = 0.0
collector_idx = []
for i in range(N):
    u  = 0.5 * (u_nodes[i] + 1)
    wu = 0.5 * u_wts[i]
    for j in range(N):
        v  = 0.5 * (v_nodes[j] + 1)
        wv = 0.5 * v_wts[j]

        # Compute point on patch
        p = p0 + u*du_vec + v*dv_vec

        # Direction from patch to source
        dir_sp = source_pos - p
        r2     = np.dot(dir_sp,dir_sp)
        if r2 <= 1e-12:
            continue

        r      = np.sqrt(r2)
        wi     = dir_sp / r  # incoming direction at patch
        # Cosine term at receiver
        cos_tr = np.dot(n_p, wi)
        if cos_tr <= 0:
            continue

        # Then each query is:
        idx_i = tree_i.query(wi, k=1)[1]
        Li = L_i_data[idx_i]
        collector_idx.append(idx_i)
        # Differential contribution: I * fr * cosθ_r * dA / r²
        integrand = Li * cos_tr * area / r2

        total += integrand * wu * wv

print(total)

0.00547341482228058


/var/folders/vp/j1zzmqmx3lnb6s6ydlmz13_c0000gn/T/ipykernel_7312/494939367.py:12: PyfarDeprecationWarning: 'This function will be deprecated in pyfar 0.8.0 in favor of spharpy.samplings.hyperinterpolation.'
  scattering_directions = pf.samplings.sph_lebedev(sh_order=sh_order)
/opt/miniconda3/envs/vir_spec/lib/python3.11/site-packages/pyfar/classes/coordinates.py:276: PyfarDeprecationWarning: 'This function will be deprecated in pyfar 0.8.0 in favor of spharpy.samplings.SamplingSphere.'
  self.sh_order = sh_order
/var/folders/vp/j1zzmqmx3lnb6s6ydlmz13_c0000gn/T/ipykernel_7312/494939367.py:14: PyfarDeprecationWarning: 'This function will be deprecated in pyfar 0.8.0 in favor of spharpy.samplings.calculate_sampling_weights.'
  scattering_directions.weights = pf.samplings.calculate_sph_voronoi_weights(scattering_directions)
/opt/miniconda3/envs/vir_spec/lib/python3.11/site-packages/pyfar/samplings/spatial.py:130: PyfarDeprecationWarning: 'This function will be deprecated in pyfar 0.8.0 in f

In [ ]:
### PLOTTER
import matplotlib.pyplot as plt
%matplotlib ipympl

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')

# Plot the sampled directions
sc = ax.scatter(
    shifted_coord[collector_idx, 0],
    shifted_coord[collector_idx, 1],
    shifted_coord[collector_idx, 2], cmap='viridis', label='Sampled Directions'
)

ax.scatter(
    shifted_coord[:, 0],
    shifted_coord[:, 1],
    shifted_coord[:, 2], s=1, color='green', cmap='viridis', label='Sampled Directions'
)

# Plot the source position
ax.scatter(
    source_pos[0], source_pos[1], source_pos[2],
    color='red', s=100, marker='*', label='Source Position'
)
# Plot the patch as a rectangle
patch = np.vstack([patch_pts, patch_pts[0]])  # close the loop
ax.plot(patch[:, 0], patch[:, 1], patch[:, 2], color='black', linewidth=2, label='Patch')
ax.legend()
plt.colorbar(sc, ax=ax, label='L_i_data')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_zlim(0, 1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('Sampling Directions Colored by L_i_data')
plt.show()

#scattering_directions.show()